In [54]:
from bs4 import BeautifulSoup   # For parsing HTML/XML documents
import requests                # For making HTTP requests to fetch web pages
from urllib.parse import urlparse
import random
import time                   #For human like interaction
from time import sleep

In [55]:
robots_url = "https://www.enterprise.com/robots.txt"

response = requests.get(robots_url)

if response.status_code == 200:
    print(response.text)
else:
    print("Failed to fetch robots.txt. Status code:", response.status_code)

# Go to the site itselt to see its robot.txt file

Failed to fetch robots.txt. Status code: 403


In [56]:
# Hardcoded robots.txt rules
robots_txt = {
    "*": [
        "/en/reserve.html",
        "/es/reserve.html",
        "/content/ecom/",
        "/corporate-accounts",
        "*/beta",
        "/en/reserve/check-in.html",
        "/es/reserve/check-in.html",
        "/content/ecom/en/staging",
        "/content/ecom/es/staging",
        "/en/reserve/check-out.html",
        "/es/reserve/check-out.html",
        "/en/car-rental/campaigns/locations-near-me.html",
        "/en/reserve/modify-rental.html",
        "/en/reserve/univ-prep.html",
        "/es/reserve/modify-rental.html",
        "/en/account/account-activity.html",
        "/en/account/request-missing-points.html",
        "/es/account/account-activity.html",
        "/es/account/request-missing-points.html",
        "/en/reserve/view-modify-cancel/rental-search.html",
        "/es/reserve/view-modify-cancel/rental-search.html",
        "/en/communication-preferences.html",
        "/en/communication-preferences/unsubscribe.html",
        "/es/communication-preferences.html",
        "/es/communication-preferences/unsubscribe.html"
    ]
}

def is_allowed(url):
    # Parse the path from the URL
    from urllib.parse import urlparse
    path = urlparse(url).path

    # Check if the path is disallowed for the given user-agent
    for disallowed_url in robots_txt["*"]:
        if path.startswith(disallowed_url):
            return False

    return True

def GET_UA():
    uastrings = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
        "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"
    ]
    return random.choice(uastrings)

def parse_url(url):
    headers = {'User-Agent': GET_UA()}
    response_code = None

    try:
        response = requests.get(url, headers=headers)
        response_code = response.status_code

    except Exception as e:
        print("Error:", str(e))

    return response_code

# Modify the proxy_request function to use a proper endpoint and include authentication
def proxy_request(url):
    headers = {'User-Agent': GET_UA()}
    proxy_ip = "50.168.72.119"
    proxy_port = 31147
    proxies = {
    "http": f"http://{proxy_ip}:{proxy_port}",
    "https": f"http://{proxy_ip}:{proxy_port}"
    }
    try:
        # Send a request using the proxy
        response = requests.get(url, headers=headers, proxies=proxies)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            return soup
        else:
            print("Request failed with status code:", response.status_code)
            return None

    except Exception as e:
        print("An error occurred:", str(e))
        return None


In [58]:
# URL to scrape
url = "https://www.enterprise.com/en/home.html"

# Send a GET request to fetch the HTML content
response = proxy_request(url)

# Check if the request was successful (status code 200)
if response.status_code == 200 and is_allowed("https://www.enterprise.com/en/home.html"):
    # Print the HTML content
    print(response.text, "AND URL is allowed to be crawled.")
else:
    print("Failed to fetch URL. Status code:", response.status_code, " OR URL is not allowed to be crawled.")


An error occurred: HTTPSConnectionPool(host='www.enterprise.com', port=443): Max retries exceeded with url: /en/home.html (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E4EF30E020>, 'Connection to 50.168.72.119 timed out. (connect timeout=None)')))


AttributeError: 'NoneType' object has no attribute 'status_code'

In [59]:
url = ["https://www.enterprise.com", 'https://www.enterprise.com/en/home.html', 'https://www.enterprise.com/en/reserve.html#car_select']

for urls in url:
    response_code = parse_url(urls)
    print(urls)
    if response_code == 200 and is_allowed(urls):
        print("Response Status Code:", response_code, " AND URL is allowed to be crawled.")
    else:   
        print("Failed to fetch URL. Response status code:", response_code, " OR URL is not allowed to be crawled.")


https://www.enterprise.com
Response Status Code: 200  AND URL is allowed to be crawled.
https://www.enterprise.com/en/home.html
Response Status Code: 200  AND URL is allowed to be crawled.
https://www.enterprise.com/en/reserve.html#car_select
Failed to fetch URL. Response status code: 200  OR URL is not allowed to be crawled.


In [61]:
start = time.time()

response_code = parse_url(url[1])
print(response_code, "=> FIRST PHASE -> COMPLETE")

# Check if the request was successful (status code 200)
if response_code == 200:
    # Send another request to fetch the full response
    response = requests.get(url[1])
    # Parse the HTML content
    time.sleep(random.randint(2, 6)) 
    soup = BeautifulSoup(response.content, "html.parser")
    print("")
    print(soup)
    print("")

    # Find the location input field
    location_input = soup.find("input", class_="rs-input__field selected")
    time.sleep(random.randint(2, 6))  # Introduce a random delay

    # Check if the location input field exists
    if location_input:
        # Set the value of the location input field (e.g., "New York JFK International Airport")
        location_input["value"] = "New York JFK International Airport"
    else:
        print("Location input field not found.")

    # Find the location item in the list and click on it
    location_item = soup.find("li", id="location-1018781")
    if location_item:
        location_item.click()
        time.sleep(2.6)  # Introduce a delay to simulate human-like interaction
    else:
        print("Location item not found.")

    # Find the button and extract its URL or form action
    button = soup.find("button", class_="cta cta--primary cta--extra-large cta--fullWidth cta--noMargin")
    button_url = button.get("href")  # If the button is an anchor element
    button_action = button.get("formaction")  # If the button is a submit button within a form

    print("First.FIVE IF condition is successful")
    # Send a POST request to submit the form data (if applicable)
    if button_url:
        response = requests.post(button_url, data={"location": "New York JFK International Airport"})
        print("SECOND IF condition is successful")
    elif button_action:
        response = requests.post(url, data={"location": "New York JFK International Airport"})
        print("SECOND IF condition is successful")

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the next page
        next_page_soup = BeautifulSoup(response.content, "html.parser")
        print("SUCCESSFUL")
        # Extract relevant information from the next page (if needed)
        # ...

        # Print or process the extracted information
        # ...
else:
    print("Failed to fetch URL. Status code:", response_code)

end = time.time()
print("Time taken:", end - start)


200 => FIRST PHASE -> COMPLETE

<html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://www.enterprise.com/en/home.html" on this server.<p>
Reference #18.6c5d3a17.1711404007.4e9b84
<p>https://errors.edgesuite.net/18.6c5d3a17.1711404007.4e9b84</p>
</p></body>
</html>


Location input field not found.
Location item not found.


AttributeError: 'NoneType' object has no attribute 'get'

In [62]:
# To battle this, we are using PROXY
# But as long it is a free proxy, there is less chance of scraping the website
# Also idk why but my lap is hanging... IG because of 9+ desktop sites being opened my lap
# IF ALLOWED SELENIUM, then I would have been able to scrape the website

url = "https://www.enterprise.com/en/home.html"
soup = proxy_request(url)
if soup:
    print("Page content retrieved successfully:", soup)
else:
    print("Failed to retrieve page content.")

An error occurred: HTTPSConnectionPool(host='www.enterprise.com', port=443): Max retries exceeded with url: /en/home.html (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E4EF23C5E0>, 'Connection to 50.168.72.119 timed out. (connect timeout=None)')))
Failed to retrieve page content.


In [63]:
# Define the proxy server details
proxy_ip = "64.227.4.90"
proxy_port = 31147

# URL to access through the proxy
url = "https://www.enterprise.com/en/home.html"

# Set up the proxy configuration
proxies = {
    "http": f"http://{proxy_ip}:{proxy_port}",
    "https": f"http://{proxy_ip}:{proxy_port}"
}

# Send a request using the proxy
try:
    response = requests.get(url, proxies=proxies)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        print("Request successful!")

        # Introduce additional delay
        time.sleep(random.uniform(3, 6))

        # Find the location input field
        location_input = soup.find("input", class_="rs-input__field selected")

        # Check if the location input field exists
        if location_input:
            # Set the value of the location input field
            location_input["value"] = "New York JFK International Airport"
        else:
            print("Location input field not found.")

        # Find the location item in the list and click on it
        location_item = soup.find("li", id="location-1018781")
        if location_item:
            location_item.click()
            # Introduce additional delay
            time.sleep(2.6)
        else:
            print("Location item not found.")

    else:
        print("Request failed with status code:", response.status_code)

except Exception as e:
    print("An error occurred:", str(e))


An error occurred: HTTPSConnectionPool(host='www.enterprise.com', port=443): Max retries exceeded with url: /en/home.html (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E4EF30CDF0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')))
